In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f reddit-technology-toplevel-title.csv.xz || wget  https://github.com/heiseacademy/ml-python/raw/main/02-ml-intro/reddit-technology-toplevel-title.csv.xz")

In [ ]:
import pandas as pd
docs = pd.read_csv("reddit-technology-toplevel-title.csv.xz", parse_dates=["created_utc"])

In [ ]:
docs

In [ ]:
docs.set_index("created_utc").resample("QE").count().plot()

In [ ]:
stop_words= {"'d", "'ll", "'m", "'re", "'s", "'ve", 'a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 
             'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 
             'amongst', 'amount', 'amp', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 
             'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 
             'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 
             'beyond', 'blog', 'body', 'both', 'bottom', 'but', 'buy', 'buycheap', 'by', 'ca', 'call', 'can', 'cannot', 
             'case', 'change', 'co', 'com', 'could', 'create', 'delete', 'did', 'do', 'does', 'doing', 'done', 'down', 
             'download', 'drive', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'email', 
             'empty', 'enough', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 
             'fifteen', 'fifty', 'first', 'five', 'fix', 'for', 'former', 'formerly', 'forty', 'four', 'from', 'front', 
             'full', 'further', 'get', 'give', 'go', 'good', 'had', 'has', 'have', 'he', 'help', 'hence', 'her', 'here', 
             'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 
             'http', 'https', 'hundred', 'i', 'if', 'in', 'indeed', 'into', 'is', 'it', 'its', 'itself', 'just', 'keep', 
             'last', 'late', 'latter', 'latterly', 'least', 'less', 'll', 'look', 'made', 'make', 'many', 'market', 'may', 
             'me', 'meanwhile', 'message', 'might', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 
             'must', 'my', 'myself', "n't", 'name', 'namely', 'need', 'neither', 'never', 'nevertheless', 'new', 'news', 
             'next', 'nine', 'no', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'number', 'n‘t', 
             'n’t', 'of', 'off', 'often', 'on', 'once', 'one', 'online', 'only', 'onto', 'or', 'other', 'others', 
             'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'page', 'part', 'pass', 'per', 'perhaps', 
             'please', 'post', 'put', 'question', 'quite', 'rather', 're', 'really', 'reddit', 'regarding', 'remove', 
             'review', 'same', 'say', 'search', 'see', 'seem', 'seemed', 'seeming', 'seems', 'self', 'send', 'serious', 
             'several', 'she', 'should', 'show', 'side', 'since', 'site', 'six', 'sixty', 'so', 'some', 'somehow', 
             'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'support', 'take', 'ten', 
             'test', 'text', 'than', 'that', 'the', 'their', 'them', 'themselves', 'then', 'thence', 'there', 
             'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'third', 'this', 'those', 
             'though', 'three', 'through', 'throughout', 'thru', 'thus', 'time', 'to', 'together', 'too', 'top', 
             'toward', 'towards', 'twelve', 'twenty', 'two', 'under', 'unless', 'unlock', 'until', 'up', 'upon', 
             'us', 'use', 'used', 'using', 'various', 've', 'very', 'via', 'video', 'was', 'watch', 'way', 'we', 'well', 
             'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 
             'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 
             'whom', 'whose', 'why', 'will', 'with', 'within', 'without', 'work', 'would', 'yet', 'you', 'your', 
             'yours', 'yourself', 'yourselves', '‘d', '‘ll', '‘m', '‘re', '‘s', '‘ve', '’d', '’ll', '’m', '’re', 
             '’s', '’ve'}

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=10, stop_words=list(stop_words))
tfidf_vectors = tfidf_vectorizer.fit_transform(docs["title"].map(str))
tfidf_vectors

In [ ]:
tfidf_vectors.data.nbytes

In [ ]:
docs.groupby("flair").agg({"title": "count"}).sort_values("title", ascending=False).head(20)

In [ ]:
docs["target"] = 0
docs.loc[docs["flair"] == "Artificial Intelligence", "target"] = 1

In [ ]:
ai = docs[docs["flair"] == "Artificial Intelligence"]
len(ai)

In [ ]:
non_ai = docs[docs["flair"] != "Artificial Intelligence"]
len(non_ai)

In [ ]:
data = pd.concat([ai, non_ai.sample(n = len(ai), random_state=42)])
data

In [ ]:
ai_tfidf_vectors = tfidf_vectorizer.transform(data["title"].map(str))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ai_tfidf_vectors, data['target'], test_size = 0.25, random_state = 42)

In [ ]:
from sklearn.linear_model import SGDClassifier
ai_svm = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
ai_svm.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, ai_svm.predict(X_test)))